# StarMallow Goals/examples

## Detect Schema from marshmallow-dataclass

By default, read from JSON body and return JSON body

In [ ]:
import marshmallow.fields as mf
from marshmallow_dataclass import dataclass as ma_dataclass
from starmallow import APIRouter
from starmallow.params import (
  Header, 
  Body, 
  Query,
  Path,
  Cookie,
  Form,
  File,
)

@ma_dataclass
class CreateRequest:
  pass

@ma_dataclass
class CreateResponse:
  pass


router = APIRouter(prefix='/user', name='user')

## Read from json body 
@router.post('/create', status_code=202)
async def create_user(
  create_request: CreateRequest,
  create_request_json: CreateRequest = Body(None),
  create_request_query: CreateRequest = Query(None),
  create_request_path: CreateRequest = Path(None),
  create_request_header: CreateRequest = Header(None),
  create_request_cookie: CreateRequest = Cookie(None),
  create_request_form: CreateRequest = Form(None),
  create_request_file: CreateRequest = File(None),

  # Custom schema
  create_request_custom: CreateRequest = Body(None, model=CreateRequest.Schema),

  # Allow usign marshmallow fields directly as well?
  my_int: int = Body(None),
  email: str = Body(None, schema=mf.Email())
) -> CreateResponse:
  pass


## Playground

In [2]:
from typing import Any, Optional, Union

import marshmallow as ma
import marshmallow.fields as mf


class FieldInfo:

    def __init__(
        self,
        default: Any,
        *,
        deprecated: Optional[bool] = None,
        include_in_schema: bool = True,
        model: Union[ma.Schema, mf.Field] = None,
    ) -> None:
        self.default = default
        self.deprecated = deprecated
        self.include_in_schema = include_in_schema
        self.model = model


class Path(FieldInfo):
    pass


class Query(FieldInfo):
    pass


class Header(FieldInfo):
    pass


class Cookie(FieldInfo):
    pass


class Body(FieldInfo):
    pass


class Form(FieldInfo):
    pass


class File(FieldInfo):
    pass


In [15]:

from typing import Any, Callable, TypeVar, Generic, Optional, _SpecialForm

import marshmallow.fields as mf
from marshmallow_dataclass import dataclass as ma_dataclass

@ma_dataclass
class CreateRequest:
  my_int2: int

@ma_dataclass
class CreateResponse:
  pass


    
@ma_dataclass
class Foobar:
  foo: str
  bar: str

def test(foobar: Foobar) -> None:
  print(foobar.foo)
  print(foobar.bar)

test(Foobar(foo='FOO', bar='bar'))


async def create_user(
  create_request: CreateRequest,
  create_request_json: CreateRequest = Body(...),
  create_request_query: CreateRequest = Query(...),
  create_request_path: CreateRequest = Path(...),
  create_request_header: CreateRequest = Header(...),
  create_request_cookie: CreateRequest = Cookie(...),
  create_request_form: CreateRequest = Form(...),
  
  # Custom schema
  create_request_custom: CreateRequest = Body(..., model=CreateRequest.Schema),

  # Allow usign marshmallow fields directly as well?
  my_int: Optional[int] = Body(5),
  email: str = Body(..., model=mf.Email()),

  my_int2: int = Header(...),

  optional: Optional[CreateRequest] = Body(None),
) -> CreateResponse:
  pass


FOO
bar


In [18]:
import inspect

parameters = dict(inspect.signature(create_user).parameters)

parameters['create_request'].default
parameters['create_request_json'].default


In [29]:
import marshmallow as ma
import marshmallow.fields as mf

t = mf.Integer(required=True)

try:
  t.deserialize('1d', 'foobar', {'foobar': '1d'})
except Exception as e:
  ex = e

In [30]:
ex.__dict__

{'messages': ['Not a valid integer.'],
 'field_name': '_schema',
 'data': None,
 'valid_data': None,
 'kwargs': {}}

In [ ]:
# Validation response
## FastAPI
{
    "detail": [
        {
            "loc": [
                "path",
                "item_id"
            ],
            "msg": "value is not a valid integer",
            "type": "type_error.integer"
        }
    ],
    "error": "ValidationError",
    "status_code": 422
}

## Flask-Smorest
{
    "code": 422,
    "errors": {
        "json": {
            "columns": [
                "Missing data for required field."
            ],
            "name": [
                "Missing data for required field."
            ],
            "order_columns": [
                "Missing data for required field."
            ],
            "sql_query": [
                "Missing data for required field."
            ],
            "table_names": [
                "Missing data for required field."
            ]
        }
    },
    "status": "Unprocessable Entity"
}

In [32]:
import marshmallow as ma
import marshmallow.fields as mf
from dataclasses import dataclass

@dataclass
class Foobar:
  a: str
  b: int

class FoobarSchema(ma.Schema):
  a = mf.String()
  b = mf.Integer()


f = Foobar('A', 2)
FoobarSchema().dump(f)

{'b': 2, 'a': 'A'}

## Formdata request with multiple content-types

In [ ]:
import http.client
import mimetypes
from codecs import encode

conn = http.client.HTTPConnection("127.0.0.1", 8000)
dataList = []
boundary = 'wL36Yn8afVp8Ag7AmP8qZ0SA4n1v9T'
dataList.append(encode('--' + boundary))
dataList.append(encode('Content-Disposition: form-data; name=foobar_json;'))

dataList.append(encode('Content-Type: {}'.format('application/json')))
dataList.append(encode(''))

dataList.append(encode('{"foo": "bar"}'))
dataList.append(encode('--' + boundary))
dataList.append(encode('Content-Disposition: form-data; name=foobar;'))

dataList.append(encode('Content-Type: {}'.format('text/plain')))
dataList.append(encode(''))

dataList.append(encode("foo"))
dataList.append(encode('--' + boundary))
dataList.append(encode('Content-Disposition: form-data; name=file; filename={0}'.format('/C:/Users/maste/Downloads/new_user_credentials.csv')))

fileType = mimetypes.guess_type('/C:/Users/maste/Downloads/new_user_credentials.csv')[0] or 'application/octet-stream'
dataList.append(encode('Content-Type: {}'.format(fileType)))
dataList.append(encode(''))

with open('C:/Users/maste/Downloads/new_user_credentials.csv', 'rb') as f:
  dataList.append(f.read())
dataList.append(encode('--'+boundary+'--'))
dataList.append(encode(''))
body = b'\r\n'.join(dataList)
payload = body
headers = {
   'Content-type': 'multipart/form-data; boundary={}'.format(boundary) 
}
conn.request("POST", "/query/run", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))